### Install pandasql to run sql commands on pandas dataframe

In [59]:
!pip install -U pandasql

In [60]:
import pandas as pd
import numpy as np #for array computations
from pandasql import sqldf #Importing panda sql

# 1. Reading all provided dataframes

In [61]:
'''
To read the excel files please store the code and data in same directory OR change the file directory of files below
'''
kw_performance = pd.read_excel('KW_Performance_L120D.XLSX') 
kw_performance.columns= kw_performance.columns.str.strip().str.lower() #Returning lowercased values of column names for easier calls
kw_performance.rename(columns={'kw id': 'kw_id'},inplace=True) #Renaming columns to replace space with underscore (easier for sql calls)
kw_performance['kw_id'] = kw_performance['kw_id'].apply(str) #changing kw_id to str for easier use in later functions
kw_performance['cvr']=kw_performance['conversions']/kw_performance['clicks']
kw_performance.sort_values(['conversions'],ascending=False) #calls the first 10 rows of dataframe


,kw_id,impressions,clicks,cost,conversions,cvr
4,534068489,32940,520,2872.377114,43,0.082692
9,486105700,23800,900,1937.821743,39,0.043333
16,323755079,44800,580,856.967719,35,0.060345
23,158812592,36020,670,1330.067232,35,0.052239
22,139891896,47500,1580,2449.864745,35,0.022152
...,...,...,...,...,...,...
2232,353169625,10440,920,1002.343448,0,0.000000
2231,410835656,8860,760,1906.898820,0,0.000000
2230,374130275,7030,480,1738.697541,0,0.000000
2229,278635170,3770,430,3145.022414,0,0.000000


In [62]:
inventory_current_onsite = pd.read_excel('Inventory_Current_Onsite.XLSX')
inventory_current_onsite.columns= inventory_current_onsite.columns.str.strip().str.lower()
inventory_current_onsite.tail(10) #calls the last 10 rows of dataframe

,make,model,year,currentonsiteinventory
105,Kia,Sorento,2011,8
106,Cadillac,SRX,2011,5
107,Chevrolet,Camaro,2011,10
108,Chevrolet,Equinox,2011,7
109,GMC,Acadia,2015,11
110,Kia,Forte,2015,5
111,Nissan,LEAF,2015,3
112,GMC,Acadia,2011,6
113,Kia,Forte,2011,6
114,Nissan,LEAF,2011,3


In [63]:
inventory_historical = pd.read_excel('Inventory_Historical.XLSX')
inventory_historical.columns= inventory_historical.columns.str.strip().str.lower()
inventory_historical

,make,model,year,histavginv
0,Toyota,Camry,2013,143
1,Kia,Soul,2013,140
2,Honda,Civic,2013,124
3,Toyota,Corolla,2013,120
4,Hyundai,Elantra,2013,114
...,...,...,...,...
110,Kia,Forte,2015,7
111,Nissan,LEAF,2015,6
112,GMC,Acadia,2011,7
113,Kia,Forte,2011,6


In [64]:
kw_attributes = pd.read_excel('KW_Attributes.XLSX')
kw_attributes.columns= kw_attributes.columns.str.strip().str.lower()
kw_attributes.rename(columns={'kw id': 'kw_id','ad group': 'ad_group','match type': 'match_type','quality score': 'quality_score','est first pos. bid': 'est_first_pos_bid','est top of page bid': 'est_top_page_bid'},inplace=True)
kw_attributes['kw_id'] = kw_attributes['kw_id'].apply(str)
kw_attributes

,campaign,ad_group,keyword,kw_id,match_type,quality_score,est_first_pos_bid,est_top_page_bid
0,SRCH-I-ATL-TOTL,SRCH-I-ATL-MK_Toyota-MO_Camry-YR_12,+2012 +toyota +camry,371462857,Broad,10,9.979746,8.316455
1,SRCH-I-ATL-TOTL,SRCH-I-ATL-MK_Toyota-MO_Camry-YR_14,+2014 +toyota +camry +for +sale,540282177,Broad,10,11.620050,9.683375
2,SRCH-I-ATL-TOTL,SRCH-I-ATL-MK_Toyota-MO_Camry-YR_15,+2015 +toyota +camry +for +sale,518188218,Broad,9,8.635562,7.196302
3,SRCH-I-ATL-TOTL,SRCH-I-ATL-MK_Kia-MO_Soul-YR_15,+2015 +kia +soul,698853859,Broad,10,10.979037,9.149198
4,SRCH-I-ATL-TOTL,SRCH-I-ATL-MK_Toyota-MO_Camry-YR_13,+toyota +camry +2013,534068489,Broad,7,10.474498,5.844252
...,...,...,...,...,...,...,...,...
3445,SRCH-I-SFO-TOTL,SRCH-I-SFO-MK_Toyota-MO_Prius-YR_15,+2015 +toyota +prius +for +sale,193408385,Broad,3,11.025667,2.482643
3446,SRCH-I-SFO-TOTL,SRCH-I-SFO-MK_Toyota-MO_Prius-YR_15,2015 toyota prius,322574626,Exact,4,11.734692,9.778910
3447,SRCH-I-SFO-TOTL,SRCH-I-SFO-MK_Toyota-MO_Prius-YR_15,toyota prius 2015,849508696,Exact,7,6.203357,2.831516
3448,SRCH-I-SFO-TOTL,SRCH-I-SFO-MK_Toyota-MO_Prius-YR_15,+toyota +prius +2015,836776515,Broad,8,10.928894,9.107412


In [65]:
make_model_ARS = pd.read_excel('Make_Model_ARS.XLSX')
make_model_ARS.columns= make_model_ARS.columns.str.strip().str.lower()
make_model_ARS.rename(columns={'make model': 'make_model'},inplace=True)
make_model_ARS

,make_model,make,model,ars
0,Toyota Camry,Toyota,Camry,930.517547
1,Kia Soul,Kia,Soul,975.778126
2,Honda Civic,Honda,Civic,1120.123183
3,Toyota Corolla,Toyota,Corolla,861.917229
4,Hyundai Elantra,Hyundai,Elantra,959.873535
5,Ford Fusion,Ford,Fusion,1060.953889
6,GMC Terrain,GMC,Terrain,1077.603723
7,Toyota Prius,Toyota,Prius,1088.587793
8,Honda Accord,Honda,Accord,913.153110
9,Hyundai Sonata,Hyundai,Sonata,1004.281070


In [66]:
import warnings
warnings.filterwarnings('ignore') #to remove warning pop-ups

# 2. Aggregate KW data to get CVR for AG, Mk/Mo/Yr, Mk/Mo and Mkt data

### 2.1 Extracting market, make, model, year from ad_group column in kw_attributes

In [67]:
kw_attributes2 = kw_attributes[['ad_group','kw_id']]
kw_attributes2[['d','r','market','make','model','yr']] = kw_attributes2['ad_group'].str.split('-', expand=True)
kw_attributes2 = kw_attributes2[['kw_id','market','make','model','yr']]
kw_attributes2['kw_id'] = kw_attributes2['kw_id'].astype(str)
splits = [kw_attributes2[col].str.split(pat='_', expand=True).add_prefix(col) for col in kw_attributes2.columns]
clean_df = pd.concat(splits, axis=1)
clean_df = clean_df.drop(['make0','model0','yr0'], axis=1)
clean_df.rename(columns={'make1': 'make','model1': 'model','yr1': 'yr','kw_id0': 'kw_id','market0': 'market'},inplace=True)
clean_df.head(5)

,kw_id,market,make,model,yr
0,371462857,ATL,Toyota,Camry,12
1,540282177,ATL,Toyota,Camry,14
2,518188218,ATL,Toyota,Camry,15
3,698853859,ATL,Kia,Soul,15
4,534068489,ATL,Toyota,Camry,13


In [68]:
'''
The sqldfmethod is used to query the Dataframes and it requires 2 inputs:
            a) The SQL query string
            b) globals()or locals() function
'''
pysqldf = lambda q: sqldf(q, globals()) 
query='select market,keyword from clean_df c join kw_attributes kw_a on c.kw_id=kw_a.kw_id'
mkt_kw=pysqldf(query)
mkt_kw.head(5)

,market,keyword
0,ATL,+2012 +toyota +camry
1,ATL,+2014 +toyota +camry +for +sale
2,ATL,+2015 +toyota +camry +for +sale
3,ATL,+2015 +kia +soul
4,ATL,+toyota +camry +2013


### 2.2)   Aggregate KW data to get AG_CVR

In [69]:
query='select * from kw_performance perf join ( select * from kw_attributes ) attr on perf.kw_id = attr.kw_id'
adg_cvr_t=pysqldf(query)
adg_cvr_t=adg_cvr_t.groupby(['ad_group'])['clicks','conversions'].sum().reset_index()
adg_cvr_t['cvr']=adg_cvr_t['conversions']/adg_cvr_t['clicks']
adg_cvr_t.head(5)

,ad_group,clicks,conversions,cvr
0,SRCH-I-ATL-MK_Buick-MO_Verano-YR_11,1950,2,0.001026
1,SRCH-I-ATL-MK_Buick-MO_Verano-YR_12,980,6,0.006122
2,SRCH-I-ATL-MK_Buick-MO_Verano-YR_13,2530,23,0.009091
3,SRCH-I-ATL-MK_Buick-MO_Verano-YR_14,2720,7,0.002574
4,SRCH-I-ATL-MK_Buick-MO_Verano-YR_15,3850,3,0.000779


### 2.3)   Aggregate KW data to get mk_mo_yr_CVR

In [70]:
query='select * from kw_performance perf join ( select * from clean_df ) attr on perf.kw_id = attr.kw_id'
mk_mo_yr_cvr_t=pysqldf(query)
mk_mo_yr_cvr_t=mk_mo_yr_cvr_t.groupby(['make','model','yr'])['clicks','conversions'].sum().reset_index()
mk_mo_yr_cvr_t['cvr']=mk_mo_yr_cvr_t['conversions']/mk_mo_yr_cvr_t['clicks']
mk_mo_yr_cvr_t.head(5)

,make,model,yr,clicks,conversions,cvr
0,Buick,Verano,11,9260,6,0.000648
1,Buick,Verano,12,8250,32,0.003879
2,Buick,Verano,13,13420,51,0.003800
3,Buick,Verano,14,15360,22,0.001432
4,Buick,Verano,15,19230,11,0.000572


### 2.4)   Aggregate KW data to get mk_mo_CVR

In [71]:
query='select * from kw_performance perf join ( select * from clean_df ) attr on perf.kw_id = attr.kw_id'
mk_mo_cvr_t=pysqldf(query)
mk_mo_cvr_t=mk_mo_cvr_t.groupby(['make','model'])['clicks','conversions'].sum().reset_index()
mk_mo_cvr_t['cvr']=mk_mo_cvr_t['conversions']/mk_mo_cvr_t['clicks']
mk_mo_cvr_t.head(5)

,make,model,clicks,conversions,cvr
0,Buick,Verano,65520,122,0.001862
1,Cadillac,SRX,52550,92,0.001751
2,Chevrolet,Camaro,61010,97,0.001590
3,Chevrolet,Cruze,80180,200,0.002494
4,Chevrolet,Equinox,60720,96,0.001581


### 2.5)   Aggregate KW data to get mkt_CVR

In [72]:
query='select * from kw_performance perf join ( select * from clean_df ) attr on perf.kw_id = attr.kw_id'
mkt_cvr_t=pysqldf(query)
mkt_cvr_t=mkt_cvr_t.groupby(['market'])['clicks','conversions'].sum().reset_index()
mkt_cvr_t['cvr']=mkt_cvr_t['conversions']/mkt_cvr_t['clicks']
mkt_cvr_t.head(5)

,market,clicks,conversions,cvr
0,ATL,362330,1759,0.004855
1,CHI,376910,490,0.001300
2,DAL,378230,1457,0.003852
3,NYC,360000,909,0.002525
4,SFO,379000,297,0.000784


### 2.6)   Aggregate Kw_id data to get kw_CVR

In [73]:
query='select * from kw_performance perf join ( select * from kw_attributes ) attr on perf.kw_id = attr.kw_id'
kw_cvr_t=pysqldf(query)
kw_cvr_t=kw_cvr_t.groupby(['keyword'])['clicks','conversions'].sum().reset_index()
kw_cvr_t['cvr']=kw_cvr_t['conversions']/kw_cvr_t['clicks']
kw_cvr_t.sort_values(['conversions'],ascending=False)

,keyword,clicks,conversions,cvr
332,+toyota +camry +2013,4180,78,0.018660
127,+2013 +kia +soul +for +sale,4050,74,0.018272
126,+2013 +kia +soul,4960,66,0.013306
478,2013 toyota camry for sale,5710,64,0.011208
677,toyota camry 2013,4320,63,0.014583
...,...,...,...,...
325,+nissan +leaf +2011,1190,0,0.000000
328,+nissan +leaf +2014,1880,0,0.000000
329,+nissan +leaf +2015,2170,0,0.000000
346,2011 buick verano for sale,2170,0,0.000000


# 3. Building functions for each individual bidding logic

## 3.1) Returns make,model,year for the keyword

In [74]:
def get_makemodelyear(k):
    query = 'select keyword,make,model,yr from clean_df join kw_attributes kw_a on clean_df.kw_id=kw_a.kw_id group by 1,2,3'
    ans=pysqldf(query)
    ans1 = ans.loc[ans['keyword'] == k, 'make'].values[0]
    ans2 = ans.loc[ans['keyword'] == k, 'model'].values[0]
    ans3 = ans.loc[ans['keyword'] == k, 'yr'].values[0]
#ans1=ans.query('keyword == k')['make']
    return ans1,ans2,ans3

## 3.2) Returns make_model_ARS for the make+model

In [75]:
def get_mkmo_ARS(mkmo):
    '''
        returns make_model_ars value for the specified make and model
    '''
    query = "Select ars from make_model_ARS where LOWER(make_model) = LOWER(\'"+ mkmo+"\');"
    
    return pysqldf(query)

## 3.3) Returns CVR for keyword,ad_group,mk_mo_yr,mk_mo,mkt depending on the input

In [76]:
def getConversionandRate(attribute, value):
    '''
        returns the no of conversions and conversion rate for the attribute
    '''
    if attribute == 'keyword':
        query = 'select * from kw_cvr_t' 
        ans=pysqldf(query)
        ans1 = ans.loc[ans['keyword'] == value[0],'conversions'].values[0]
        ans2 = ans.loc[ans['keyword']==value[0],'cvr'].values[0]

    elif attribute == 'ad_group':
        query = 'select * from adg_cvr_t'
        ans=pysqldf(query)
        ans1 = ans.loc[ans['ad_group'] == value[0],'conversions'].values[0]
        ans2 = ans.loc[ans['ad_group']==value[0],'cvr'].values[0]
    elif attribute == 'mk_mo_yr':
        query = 'select * from mk_mo_yr_cvr_t'
        ans=pysqldf(query)
        ans1 = ans.loc[(ans.make == value[0]) & (ans.model == value[1]) & (ans.yr == value[2])].iloc[0]['conversions']
        ans2=ans.loc[(ans.make == value[0]) & (ans.model == value[1]) & (ans.yr == value[2])].iloc[0]['cvr']
    elif attribute == 'mkt':
        query33 = "select * from mkt_cvr_t"
        ans=pysqldf(query33)
        ans1 = ans.loc[ans['market'] == value[0],'conversions'].values[0]
        ans2 = ans.loc[ans['market']==value[0],'cvr'].values[0]
    elif attribute == 'mk_mo':
        query11 = 'select * from mk_mo_cvr_t'
        ans=pysqldf(query11)
        df = ans.loc[(ans['make'] == value[0]) & (ans['model'] == value[1])]
        ans1 = df.iloc[0]['conversions']
        ans2 = df.iloc[0]['cvr']
    return ans1,ans2

In [77]:
kw_id='524016325'
query = "select ad_group from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\')"
adg_kw = pysqldf(query).to_numpy()[0][0]
print(adg_kw)

SRCH-I-SFO-MK_Kia-MO_Soul-YR_11


In [78]:
kw_id='524016325'
query = "select kw_id,keyword from kw_attributes"
t_k=pysqldf(query)
keyword = t_k.loc[t_k['kw_id'] == kw_id,'keyword'].values[0]
print(keyword)
kw_conv, kw_cvr = getConversionandRate('keyword', [keyword])
kw_conv, kw_cvr

2011 kia soul for sale


(13, 0.005936073059360731)

In [79]:
query = "select ad_group from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\')"
adg_kw = pysqldf(query).to_numpy()[0][0]
#print(adg_kw)
ad_conv, ad_cvr = getConversionandRate('ad_group', [adg_kw])
ad_conv, ad_cvr

(4, 0.000980392156862745)

In [80]:
make, model, year = get_makemodelyear(keyword)
mk_mo_yr_conv, mk_mo_yr_cvr = getConversionandRate('mk_mo_yr', [make, model, year])
mk_mo_yr_conv, mk_mo_yr_cvr
#make, model, year

(57, 0.003182579564489112)

In [81]:
value=[make, model]
query11 = 'select * from mk_mo_cvr_t'
ans=pysqldf(query11)
df = ans.loc[(ans['make'] == value[0]) & (ans['model'] == value[1])]
ans1 = df.iloc[0]['conversions']
ans2 = df.iloc[0]['cvr']
ans1,ans2

(665, 0.005597643097643098)

In [82]:
mk_mo_conv, mk_mo_cvr = getConversionandRate('mk_mo', [make, model])
mk_mo_conv, mk_mo_cvr

(665, 0.005597643097643098)

In [83]:
mk_mo_ars = get_mkmo_ARS(make + " " + model)
mk_mo_ars.to_numpy()

array([[975.77812606]])

## 3.4) Calculates intial bid based on bidding logic from step-1

In [84]:
kw_id='534068489'
query="select conversions,cvr from kw_performance where LOWER(kw_id) = LOWER(\'" + kw_id + "\')"
for1=pysqldf(query).to_numpy()[0]
for1[0],for1[1]


(43.0, 0.08269230769230769)

In [85]:
def new_bid_calc(kw_id):
    ''' calculated the new bid price '''
    query = "select kw_id,keyword from kw_attributes"
    t_k=pysqldf(query)
    keyword = t_k.loc[t_k['kw_id'] == kw_id,'keyword'].values[0]
    make, model, year = get_makemodelyear(keyword)
    mk_mo_ars = get_mkmo_ARS(make + " " + model)
    #print(mk_mo_ars)
    query="select conversions,cvr from kw_performance where LOWER(kw_id) = LOWER(\'" + kw_id + "\')"
    for1=pysqldf(query).to_numpy()[0]
   
    kw_conv, kw_cvr = for1[0],for1[1]
    #print(kw_cvr)
    if kw_conv > 10:
        return [(kw_cvr * mk_mo_ars).to_numpy(), 0]

    query = "select ad_group from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\')"
    adg_kw = pysqldf(query).to_numpy()[0][0]
    #print(adg_kw)
    ad_conv, ad_cvr = getConversionandRate('ad_group', [adg_kw])
    #print(ad_conv, ad_cvr)
    if kw_conv < 11 and ad_conv > 10:
        return [(ad_cvr * mk_mo_ars).to_numpy(), 0]
 
    mk_mo_yr_conv, mk_mo_yr_cvr = getConversionandRate('mk_mo_yr', [make, model, year])
    if ad_conv < 11 and mk_mo_yr_conv > 10:
        return [(mk_mo_yr_cvr * mk_mo_ars).to_numpy(), 1]

    mk_mo_conv, mk_mo_cvr = getConversionandRate('mk_mo', [make, model])
    if mk_mo_yr_conv < 11 and mk_mo_conv > 10:
        return [(mk_mo_cvr * mk_mo_ars).to_numpy(), 1]

    if mk_mo_conv < 11:
        query = "select est_first_pos_bid from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
        est_first_pos_bid = pysqldf(query)
        est_first_pos_bid=est_first_pos_bid.to_numpy()[0][0]
        return [est_first_pos_bid, 1]

In [86]:
a=new_bid_calc('266399263')
a

[array([[1.3549735]]), 1]

In [87]:
kw_id='266399263'
query="select * from clean_df where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
test=pysqldf(query)
test=test['market'].to_numpy()[0]
query="select * from mkt_cvr_t where LOWER(market) = LOWER(\'" + test + "\');"
mkt_cvr_3=pysqldf(query)
mkt_cvr_3=mkt_cvr_3['cvr'].to_numpy()[0]
mkt_cvr_3

0.0013000451036056352

In [88]:
query = "select sum(conversions)/CAST(sum(clicks)as double precision) as cvr from kw_performance;"
overall_cvr = pysqldf(query).to_numpy()
overall_cvr[0][0]

0.002645881700216001

In [89]:
kw_id='266399263'
query = "select quality_score from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
qs = pysqldf(query)
qs=qs.to_numpy()[0][0]
print(qs)
query = "select est_top_page_bid from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
est_top_page_bid = pysqldf(query)
est_top_page_bid=est_top_page_bid.to_numpy()[0][0]
print(est_top_page_bid)
query = "select est_first_pos_bid from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
est_first_pos_bid = pysqldf(query)
est_first_pos_bid=est_first_pos_bid.to_numpy()[0][0]
print(est_first_pos_bid)
print('avg',(est_first_pos_bid+est_top_page_bid)/2)

7
8.16645547774649
9.799746573295788
avg 8.98310102552114


## 3.5) Calculates the revised bid (step-2) based on intial bid

In [90]:
def revised_bid(kw_id, bid):
    ''' revises the calculated new bid price acc to Step 2 ( 1a, 1b, 1c)
    '''
    query = "select kw_id,keyword from kw_attributes"
    t_k=pysqldf(query)
    keyword = t_k.loc[t_k['kw_id'] == kw_id,'keyword'].values[0]
    
    changed_bid = bid[0][0][0]
    #print('inti',bid[0][0][0])
    make, model, year = get_makemodelyear(keyword)
    mk_mo_ars = get_mkmo_ARS(make + " " + model)

    cur_inv=inventory_current_onsite.loc[(inventory_current_onsite['make'] == make) & (inventory_current_onsite['model'] == model) & (inventory_current_onsite['year'] == 2000+int(year))].iloc[0]['currentonsiteinventory'].sum()
    hist_inv=inventory_historical.loc[(inventory_historical['make'] == make) & (inventory_historical['model'] == model) & (inventory_historical['year'] == 2000+int(year))].iloc[0]['histavginv'].sum()
    #print('inv',cur_inv,hist_inv)
    if cur_inv < hist_inv:
        percentage_reduce = ((hist_inv - cur_inv)/(hist_inv)) * 50/100
        #print(percentage_reduce)
        changed_bid = bid[0][0][0]-(percentage_reduce * bid[0][0][0])
        #print(changed_bid)
    if bid[1] == 1:
        query="select * from clean_df where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
        test=pysqldf(query)
        test=test['market'].to_numpy()[0]
        query="select * from mkt_cvr_t where LOWER(market) = LOWER(\'" + test + "\');"
        mkt_cvr_3=pysqldf(query)
        mkt_cvr_3=mkt_cvr_3['cvr'].to_numpy()[0]

        query = "select sum(conversions)/CAST(sum(clicks)as double precision) as cvr from kw_performance;"
        overall_cvr = pysqldf(query).to_numpy()
        changed_bid = changed_bid+(((mkt_cvr_3-overall_cvr[0][0] ) / 2) * changed_bid)

    query = "select quality_score from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
    qs = pysqldf(query)
    qs=qs.to_numpy()[0][0]
    #print(qs)
    query = "select est_top_page_bid from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
    est_top_page_bid = pysqldf(query)
    est_top_page_bid=est_top_page_bid.to_numpy()[0][0]
    #print(est_top_page_bid)
    query = "select est_first_pos_bid from kw_attributes where LOWER(kw_id) = LOWER(\'" + kw_id + "\');"
    est_first_pos_bid = pysqldf(query)
    est_first_pos_bid=est_first_pos_bid.to_numpy()[0][0]
    #print(est_first_pos_bid)
    #print('avg',(est_first_pos_bid+est_top_page_bid)/2)
    if qs > 7 and changed_bid > est_first_pos_bid:
        changed_bid = est_first_pos_bid
    
    elif qs < 8 and qs > 5:
    
        if changed_bid > ((est_first_pos_bid+est_top_page_bid)/2):
            changed_bid = min(changed_bid, (est_first_pos_bid+est_top_page_bid)/2)

    elif qs < 6:
        value = (est_top_page_bid * 0.9) + (est_first_pos_bid * 0.1)
        #print('val',value)
        if changed_bid > value:
            changed_bid = value

    if changed_bid > 12 or changed_bid > 12:
        changed_bid = 12.0
    return changed_bid 

In [91]:
b=revised_bid('266399263', a)
b

1.3540617095271243

## 3.6) Calculates intial final bid if kewyword match is Broad

In [92]:
def revise2_bid(bid, kw_id, final_bid):
    '''
        revises the calculated bid price acc to STep 2 (1d)
    '''
    query = "select kw_id,keyword from kw_attributes"
    t_k=pysqldf(query)
    keyword = t_k.loc[t_k['kw_id'] == kw_id,'keyword'].values[0]
    
    query = "select distinct keyword from kw_attributes where ad_group in (select distinct ad_group from kw_attributes where LOWER(keyword) = LOWER(\'" + keyword + "\') )  and LOWER(match_type) = \'exact\'"
    rows = pysqldf(query).to_numpy()
    min_group_bid = 12.0
    for keyword in rows:
        min_group_bid = min(min_group_bid, final_bid[keyword[0]])
    for keyword in rows:
        if bid > final_bid[keyword[0]]:
            bid = min_group_bid
            break
    return bid

### Example to check exact match KW within the ad_group

In [93]:
query = "select kw_id,keyword from kw_attributes"
t_k=pysqldf(query)
keyword = t_k.loc[t_k['kw_id'] == kw_id,'keyword'].values[0]

query = "select distinct keyword from kw_attributes where ad_group in (select distinct ad_group from kw_attributes where LOWER(keyword) = LOWER(\'" + keyword + "\') )  and LOWER(match_type) = \'exact\'"
rows = pysqldf(query).to_numpy()
rows
#for keyword in rows:
    
    #min_group_bid = min(min_group_bid, final_bid[keyword[0]])

array([['2011 kia forte'],
       ['2011 kia forte for sale'],
       ['kia forte 2011']], dtype=object)

## 4. Main Function stores the final bid value in csv file and calls other fns sequentially

In [95]:
def main():
   
    query = "select distinct kw_id from kw_attributes;"
    kw_ids = pysqldf(query).to_numpy()
    query = "select distinct keyword from kw_attributes;"
    keywords = pysqldf(query).to_numpy()
    final_bid = dict()
    
    for ids in kw_ids:
        new_bid = new_bid_calc(ids[0])
        final_bid[ids[0]] = revised_bid(ids[0], new_bid)
    #print(final_bid)
        
    for ids in kw_ids:
        query = "select distinct match_type from kw_attributes where LOWER(keyword) = LOWER(\'" + ids[0] + "\');"
        match = pysqldf(query)
        match['match_type']=match['match_type'].str.lower()
        if (match['match_type'] == "broad").any():
            final_bid[ids[0]] = revise2_bid(final_bid[ids[0]], ids[0], final_bid)

    file = open('final2.csv', 'w')
    file.write('{0},{1}\n'.format('keyword_id', 'bid'))
    for ids in kw_ids:
            file.write('{0},{1}\n'.format(ids[0], final_bid[ids[0]]))
    file.close()
    
if __name__ == '__main__':
    main()

# 5. Data Exploration and Analysis (Not a required part of assignment)

In [ ]:
output_bid = pd.read_csv('finall.csv')
query='select * from output_bid o left join kw_attributes kw_a on kw_a.kw_id=o.keyword_id join kw_performance perf on perf.kw_id=o.keyword_id'
merged=pysqldf(query)
merged['diff']=merged['est_first_pos_bid']-merged['bid']
query='select * from merged join ( select * from clean_df ) attr on merged.kw_id = attr.kw_id'
m2=pysqldf(query)
# mk_mo_cvr_t=mk_mo_cvr_t.groupby(['make','model'])['clicks','conversions'].sum().reset_index()
# mk_mo_cvr_t['cvr']=mk_mo_cvr_t['conversions']/mk_mo_cvr_t['clicks']
m2

In [ ]:
ma_mo_comp=m2.groupby(['make','model'])['clicks','conversions'].sum().reset_index()
ma_mo_comp['cvr']=ma_mo_comp['conversions']/ma_mo_comp['clicks']
ma_mo_comp.sort_values(by=['cvr'], ascending=False)
ma_mo_comp

In [ ]:
mkt_comp=m2.groupby(['market'])['clicks','conversions'].sum().reset_index()
mkt_comp['cvr']=mkt_comp['conversions']/camp_comp['clicks']
mkt_comp.sort_values(by=['cvr'], ascending=False)

In [ ]:
mkt_comp=m2.groupby(['market'])['clicks','conversions'].sum().reset_index()
mkt_comp['cvr']=mkt_comp['conversions']/camp_comp['clicks']
mkt_comp.sort_values(by=['cvr'], ascending=False)

In [ ]:
m2[['quality_score','est_first_pos_bid','est_top_page_bid']].describe().T

In [ ]:
import matplotlib.pyplot  as plt
import seaborn as sns
plt.figure(figsize=(12,8))
corr_var = m2[['quality_score','est_first_pos_bid','est_top_page_bid']].corr()
sns.heatmap(corr_var, annot=True, square=True, fmt='.2f', cmap='RdYlGn', vmin=-1, mask=np.triu(corr_var, 1))
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')
plt.title("Heatmap - Correlation among numerical variables", fontsize=20, fontweight='bold')
plt.show()